In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

import numpy as np
from collections import Counter
from os import walk

from IPython.display import display, HTML

### Bro/Zeek log parser and anonymiser

#### This file serves to parse Bro/Zeek logs and identify and anonymise Personally Identifiable Information (PII). As example data the bro logs of the UNSW-NB15 dataset [1] have been used.

The PII information is listed in the PII list and one line of each log file is displayed below with the PII highlighted in red.

[1] https://www.unsw.adfa.edu.au/unsw-canberra-cyber/cybersecurity/ADFA-NB15-Datasets/

In [ ]:
# Helper functions

def get_headers_types(log_type):
    headers = []
    with open("24/"+log_type) as file:
        for i in range(8):
            line = next(file).strip()
            if line.startswith("#fields"):
                headers = line.split("\t")[1:]
    return headers


def get_dataframe(log_type,seperator):
    headers = get_headers_types(log_type)
    return pd.read_csv('24/'+log_type,sep=seperator,
                       names=headers,skiprows=8,skipfooter=1,engine='python')

def highlight_col(x):
    r = 'background-color: red;'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    for info in PII:
        if info in x.columns:
            df1.loc[0,info] = r
    return df1    


In [ ]:
seperator = "[\t]"

_, _, filenames = next(walk('24/'))
filenames.sort()

# List with columns containing PII.
PII = ['id.orig_h','id.resp_h',
       'mac','assigned_ip',
       'query','answers',
       'tx_hosts','rx_hosts','md5','sha1','sha256',
       'user','password','data_channel.orig_h','data_channel.resp_h',
       'host','uri','filename','username',
       'helo','mailfrom','rcptto','date','from','to','reply_to','msg_id','in_reply_to','subject',
           'x_originating_ip','first_received','second_received','last_reply','path',
       'issuer','server_name','subject','issuer client_subject','client_issuer',
       'message',
       'certificate.serial','certificate.subject','certificate.issuer','certificate.not_valid_before',
           'certificate.not_valid_after','san.dns','san.uri','san.email','san.ip']


In [ ]:
for filename in filenames:
    print(filename)
    df_log = get_dataframe(filename,seperator).iloc[:1,:]
    display(HTML(df_log.style.apply(highlight_col, axis=None).render()))
    